In [19]:
def main():
    for article_url in article_urls:
        send_line_notify('テスト', article_url)

def send_line_notify(notification_message, url):
    """
    Lineに通知する
    """
    line_notify_token = 'LMr5ZeQuAsAz6GMxaMJyzLuhGBO3VvVIFxTwfSLUvz0'
    line_notify_api = 'https://notify-api.line.me/api/notify'
    headers = {'Authorization': f'Bearer {line_notify_token}'}
    data = {'message': url}
    requests.post(line_notify_api, headers = headers, data = data)

if __name__ == "__main__":
    main()

In [4]:
from bs4 import BeautifulSoup
import requests
import time

def scraping(topic):
    url = 'https://news.yahoo.co.jp/'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    elements = soup.select('.sc-fKgJPI.fFnFHR')
    
    articles_dict = []
    topic = topic 
    #トピックのURL取得
    for i, element in enumerate(elements):
        if topic in element.text:
            categories_num = i
    
    element = elements[categories_num]
    topic_url= element.a['href']
    URL = url + topic_url
    
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, "html.parser")
    elements = soup.select('.sc-fXazdy.UjHkE')
    elements = elements[:5]
    
    news_urls = []
    news_titles = []
    #ニュースのURLとタイトルを取得
    for element in elements:
        url = element.a['href']
        title = element.text[1:]
        news_urls.append(url)
        news_titles.append(title)
        time.sleep(1)
    
    news_text = []
    
    for news_url in news_urls:
        r = requests.get(news_url)
        soup = BeautifulSoup(r.text, "html.parser")
        elements = soup.select('.sc-dYXZXt.jzZUwL.yjSlinkDirectlink.highLightSearchTarget')

        #ニュース記事のページ数が複数のときの例外処理
        for i in range(0, 5):
            try:
                if (i == 0):
                    text = elements[0].text
                else:
                    text += elements[i].text
            except:
                pass
                
        page_num = len(soup.select('.sc-bxLXlR.kerziU'))
        #ページ数が複数のとき、ページ遷移し本文を取得
        if (page_num > 0):
            for i in range(2, page_num + 1):
                news_url_page = news_url + "?page=" + str(page_num)
                r = requests.get(news_url_page)
                soup = BeautifulSoup(r.text, "html.parser")
                elements = soup.select('.sc-dYXZXt.jzZUwL.yjSlinkDirectlink.highLightSearchTarget')
                text += elements[0].text

        print(i)
        print(text)
        
        news_text.append(text)
    
        time.sleep(1)
    
    articles = []
    for url, title, text in zip(news_urls, news_titles, news_text):
        article = {
            "title" : title,
            "URL" : url,
            "text" : text
        }
        articles.append(article)

    return articles
    

In [2]:
def clean_data(articles):
    for i in range(len(articles)):
        articles[i]['title'] = articles[i]['title'].replace('\u3000', '') 
        articles[i]['title'] = articles[i]['title'].lstrip('0123456789')
        articles[i]['text'] = articles[i]['text'].replace('\u3000', '')
        articles[i]['text'] = articles[i]['text'].replace('\n', '')

    return articles

In [5]:
articles_dict = scraping('経済')
articles = clean_data(articles_dict)

4


UnboundLocalError: cannot access local variable 'text' where it is not associated with a value

In [16]:
article_urls = []
for i, article in enumerate(articles):
    article_urls.append(articles[i]['URL'])

In [17]:
article_urls

['https://news.yahoo.co.jp/articles/b06ed103af6685e8d550cb51f787f54a9ab05540',
 'https://news.yahoo.co.jp/articles/a00b3fb475d00dd2f98ceb25c19f24c66e6454a2',
 'https://news.yahoo.co.jp/articles/5d423fdaf18612f73e3a1885e817d5245b299a58',
 'https://news.yahoo.co.jp/articles/38ad8e92cfdf6be0f5e163b1956f28a327bd9b7a',
 'https://news.yahoo.co.jp/articles/f43cb83065c79b0b6d0bc8cef20c0fe2291f865f']